# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, StratifiedKFold

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
data = pd.read_csv('/content/hasil_clustering.csv')
data.head()

,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,status_type,Cluster
0,529.0,512.0,262.0,432.0,92.0,3.0,1.0,1.0,0.0,video,0
1,150.0,0.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,photo,0
2,227.0,236.0,57.0,204.0,21.0,1.0,1.0,0.0,0.0,video,0
3,111.0,0.0,0.0,111.0,0.0,0.0,0.0,0.0,0.0,photo,0
4,213.0,0.0,0.0,204.0,9.0,0.0,0.0,0.0,0.0,photo,0


In [3]:
# Memahami Struktur Data
print("=== Memahami Struktur Data ===")
print("\nJumlah baris dan kolom dalam dataset:")
print(data.shape)

print("\nInformasi dataset:")
print(data.info())

print("\nJenis data di setiap kolom:")
print(data.dtypes)

print("\nPersebaran data pada kolom numerik:")
data.describe()

=== Memahami Struktur Data ===

Jumlah baris dan kolom dalam dataset:
(7050, 11)

Informasi dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7050 entries, 0 to 7049
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num_reactions  7050 non-null   float64
 1   num_comments   7050 non-null   float64
 2   num_shares     7050 non-null   float64
 3   num_likes      7050 non-null   float64
 4   num_loves      7050 non-null   float64
 5   num_wows       7050 non-null   float64
 6   num_hahas      7050 non-null   float64
 7   num_sads       7050 non-null   float64
 8   num_angrys     7050 non-null   float64
 9   status_type    6996 non-null   object 
 10  Cluster        7050 non-null   int64  
dtypes: float64(9), int64(1), object(1)
memory usage: 606.0+ KB
None

Jenis data di setiap kolom:
num_reactions    float64
num_comments     float64
num_shares       float64
num_likes        float64
num_loves        float

,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys,Cluster
count,7050.000000,7050.000000,7050.000000,7050.000000,7050.000000,7050.000000,7050.000000,7050.000000,7050.000000,7050.000000
mean,230.117163,224.356028,40.022553,215.043121,12.728652,1.289362,0.696454,0.243688,0.113191,0.019716
std,462.625309,889.636820,131.599965,449.472357,39.972930,8.719650,3.957183,1.597156,0.726812,0.139034
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,17.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,59.500000,4.000000,0.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,219.000000,23.000000,4.000000,184.750000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4710.000000,20990.000000,3424.000000,4710.000000,657.000000,278.000000,157.000000,51.000000,31.000000,1.000000


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [4]:
# Memisahkan Fitur (X) dan Target (y)
print("\n=== Memisahkan Fitur dan Target ===")
X = data.drop('Cluster', axis=1)  # Menghapus kolom target untuk mendapatkan fitur
y = data['Cluster']  # Mengambil kolom target

# Encoding Kolom Kategorikal
print("\n=== Encoding Kolom Kategorikal ===")
categorical_columns = X.select_dtypes(include=['object', 'bool']).columns
if not categorical_columns.empty:
    print(f"Kolom Kategorikal yang Ditemukan: {list(categorical_columns)}")
    label_encoder = LabelEncoder()
    for col in categorical_columns:
        X[col] = label_encoder.fit_transform(X[col])
        print(f"Kolom '{col}' telah di-encode.")
else:
    print("Tidak ada kolom kategorikal yang perlu di-encode.")

# Membagi Data menjadi Training dan Testing Set
print("\n=== Membagi Data ===")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Jumlah Data Training: {X_train.shape[0]} baris")
print(f"Jumlah Data Testing: {X_test.shape[0]} baris")

# Standardisasi Data Numerik
print("\n=== Standardisasi Data Numerik ===")
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
if not numerical_columns.empty:
    scaler = StandardScaler()
    X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])
    X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])
    print("Standardisasi data numerik berhasil dilakukan.")
else:
    print("Tidak ada kolom numerik yang perlu distandardisasi.")


=== Memisahkan Fitur dan Target ===

=== Encoding Kolom Kategorikal ===
Kolom Kategorikal yang Ditemukan: ['status_type']
Kolom 'status_type' telah di-encode.

=== Membagi Data ===
Jumlah Data Training: 5640 baris
Jumlah Data Testing: 1410 baris

=== Standardisasi Data Numerik ===
Standardisasi data numerik berhasil dilakukan.


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [5]:
# Fungsi untuk melatih model
def train_model(model, X_train, y_train):
    print(f"Melatih model {model.__class__.__name__}...")
    model.fit(X_train, y_train)
    print("Model selesai dilatih.")
    return model

# Pilihan Algoritma Klasifikasi
print("=== Pelatihan Model ===")

# 1. Random Forest
print("\n1. Random Forest")
rf_classifier = RandomForestClassifier(random_state=42)
rf_trained = train_model(rf_classifier, X_train, y_train)

# 2. Logistic Regression
print("\n2. Logistic Regression")
log_reg = LogisticRegression(random_state=42)
log_reg_trained = train_model(log_reg, X_train, y_train)

# Simpan model yang sudah dilatih dalam dictionary untuk evaluasi nanti
trained_models = {
    "Random Forest": rf_trained,
    "Logistic Regression": log_reg_trained
}

=== Pelatihan Model ===

1. Random Forest
Melatih model RandomForestClassifier...
Model selesai dilatih.

2. Logistic Regression
Melatih model LogisticRegression...
Model selesai dilatih.


1. **Random Forest:** Random Forest adalah algoritma ensemble learning yang menggabungkan banyak Decision Tree untuk meningkatkan akurasi dan mengurangi overfitting.

- Cara Kerja:

  a. Pembuatan Pohon Keputusan: Membuat banyak pohon keputusan menggunakan teknik bagging (Bootstrap Aggregating). Setiap pohon dilatih pada subset data yang diambil secara acak.

  b. Fitur Acak: Pada setiap split, hanya subset fitur yang dipilih secara acak yang dipertimbangkan.

  c. Voting: Prediksi akhir diambil berdasarkan voting mayoritas dari semua pohon.

- Keunggulan:

  a. Robust terhadap overfitting.
  
  b. Dapat menangani ketidakseimbangan kelas.
  
  c. Tidak membutuhkan normalisasi data.

- Kekurangan:

  a. Komputasi lebih lambat.
  
  b. Kurang interpretatif.

2. **Logistic Regression:** Logistic Regression adalah algoritma klasifikasi yang memprediksi probabilitas suatu instance termasuk dalam kelas tertentu.

- Cara Kerja:

  a. Fungsi Logistik: Menggunakan fungsi sigmoid untuk memetakan output ke rentang [0, 1].

  b. Optimasi: Meminimalkan fungsi log-loss menggunakan Gradient Descent.

  c. Decision Boundary: Jika probabilitas > 0.5, instance diklasifikasikan sebagai kelas 1; jika tidak, kelas 0.

- Keunggulan:

  a. Interpretabilitas tinggi.

  b. Efisiensi komputasi.

  c. Dapat menangani ketidakseimbangan kelas dengan class_weight='balanced'.

- Kekurangan:

  a. Mengasumsikan hubungan linear antara fitur dan target.

  b. Sensitif terhadap outlier.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [6]:
# Fungsi untuk mengevaluasi model
def evaluate_model(model, X_test, y_test):
    # Lakukan prediksi menggunakan data uji
    y_pred = model.predict(X_test)

    # Hitung metrik evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted F1-Score untuk multiclass
    precision = precision_score(y_test, y_pred, average='weighted')  # Weighted Precision
    recall = recall_score(y_test, y_pred, average='weighted')  # Weighted Recall

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Classification Report
    class_report = classification_report(y_test, y_pred)

    # Tampilkan hasil evaluasi
    print(f"\nAkurasi: {accuracy:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print("\nConfusion Matrix:")
    print(conf_matrix)
    print("\nClassification Report:")
    print(class_report)

    return y_pred

# Evaluasi semua model yang sudah dilatih
print("\n=== Evaluasi Model ===")
for model_name, model in trained_models.items():
    print(f"\nEvaluasi Model: {model_name}")
    evaluate_model(model, X_test, y_test)


=== Evaluasi Model ===

Evaluasi Model: Random Forest

Akurasi: 1.00
F1-Score: 1.00
Precision: 1.00
Recall: 1.00

Confusion Matrix:
[[1382    0]
 [   0   28]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1382
           1       1.00      1.00      1.00        28

    accuracy                           1.00      1410
   macro avg       1.00      1.00      1.00      1410
weighted avg       1.00      1.00      1.00      1410


Evaluasi Model: Logistic Regression

Akurasi: 1.00
F1-Score: 1.00
Precision: 1.00
Recall: 1.00

Confusion Matrix:
[[1381    1]
 [   0   28]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1382
           1       0.97      1.00      0.98        28

    accuracy                           1.00      1410
   macro avg       0.98      1.00      0.99      1410
weighted avg       1.00      1.00      1.00      14

- **Hasil Evaluasi Algoritma:**

  Kedua algoritma yang digunakan, yaitu Random Forest dan Logistic Regression, menunjukkan performa yang sangat baik dengan akurasi sempurna (1.00 atau 100%) pada data testing. Namun, terdapat perbedaan kecil dalam hal detail performa, terutama pada kelas minoritas (kelas 1).

  Random Forest mencatat performa sempurna pada semua metrik evaluasi, termasuk precision, recall, dan F1-score untuk kedua kelas. Confusion Matrix-nya menunjukkan tidak ada kesalahan prediksi, dengan 1382 instance kelas 0 dan 28 instance kelas 1 diprediksi dengan benar. Hal ini menunjukkan bahwa Random Forest sangat robust dan mampu menangani ketidakseimbangan kelas dengan baik.

  Di sisi lain, Logistic Regression juga mencapai akurasi sempurna, tetapi terdapat 1 false positive pada kelas 1, yang terlihat dari Confusion Matrix-nya. Meskipun recall untuk kelas 1 tetap sempurna (1.00), precision-nya sedikit lebih rendah (0.97), yang berarti ada 1 instance kelas 0 yang salah diprediksi sebagai kelas 1. Performa ini masih sangat baik, tetapi sedikit kurang akurat dibandingkan Random Forest.

  Dari segi kelebihan, Random Forest lebih unggul dalam hal robust terhadap overfitting dan performa yang konsisten pada kedua kelas. Namun, model ini lebih lambat dalam pelatihan dan kurang interpretatif. Sementara itu, Logistic Regression menawarkan kecepatan komputasi yang lebih baik dan interpretasi yang mudah melalui koefisiennya, meskipun sedikit sensitif terhadap outlier.

- **Kesimpulan:**

  Jika tujuan utama adalah akurasi tinggi dan robust terhadap overfitting, Random Forest adalah pilihan yang lebih baik. Namun, jika interpretabilitas dan kecepatan komputasi menjadi prioritas, Logistic Regression tetap menjadi opsi yang sangat baik, meskipun dengan sedikit trade-off pada performa kelas minoritas.

## **c. SMOTE untuk Imbalanced Data (Imbuhan sendiri)**

In [7]:
# Mengecek distribusi kelas sebelum SMOTE
print("Distribusi Kelas Sebelum SMOTE:")
print(y_train.value_counts())

# Menerapkan SMOTE untuk menyeimbangkan dataset
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Mengecek distribusi kelas setelah SMOTE
print("\nDistribusi Kelas Setelah SMOTE:")
print(y_train_resampled.value_counts())

Distribusi Kelas Sebelum SMOTE:
Cluster
0    5529
1     111
Name: count, dtype: int64

Distribusi Kelas Setelah SMOTE:
Cluster
0    5529
1    5529
Name: count, dtype: int64


In [8]:
# Fungsi untuk melatih model dengan data yang telah diresample
def train_resampled_model(model, X_train_resampled, y_train_resampled):
    print(f"Melatih model {model.__class__.__name__} dengan data resample...")
    model.fit(X_train_resampled, y_train_resampled)
    print("Model selesai dilatih.")
    return model

# Melatih ulang model dengan data yang telah diresample
print("\n=== Melatih Model dengan Data Resample ===")

# 1. Random Forest
print("\n1. Random Forest")
rf_classifier_resampled = RandomForestClassifier(random_state=42)
rf_resampled_trained = train_resampled_model(rf_classifier_resampled, X_train_resampled, y_train_resampled)

# 2. Logistic Regression
print("\n2. Logistic Regression")
log_reg_resampled = LogisticRegression(random_state=42)
log_reg_resampled_trained = train_resampled_model(log_reg_resampled, X_train_resampled, y_train_resampled)

# Simpan model yang sudah dilatih dengan data resample
resampled_models = {
    "Random Forest (Resampled)": rf_resampled_trained,
    "Logistic Regression (Resampled)": log_reg_resampled_trained
}


=== Melatih Model dengan Data Resample ===

1. Random Forest
Melatih model RandomForestClassifier dengan data resample...
Model selesai dilatih.

2. Logistic Regression
Melatih model LogisticRegression dengan data resample...
Model selesai dilatih.


In [9]:
# Evaluasi ulang model dengan data resample
print("\n=== Evaluasi Ulang Model dengan Data Resample ===")
for model_name, model in resampled_models.items():
    print(f"\nEvaluasi Model: {model_name}")
    evaluate_model(model, X_test, y_test)


=== Evaluasi Ulang Model dengan Data Resample ===

Evaluasi Model: Random Forest (Resampled)

Akurasi: 1.00
F1-Score: 1.00
Precision: 1.00
Recall: 1.00

Confusion Matrix:
[[1382    0]
 [   0   28]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1382
           1       1.00      1.00      1.00        28

    accuracy                           1.00      1410
   macro avg       1.00      1.00      1.00      1410
weighted avg       1.00      1.00      1.00      1410


Evaluasi Model: Logistic Regression (Resampled)

Akurasi: 1.00
F1-Score: 1.00
Precision: 1.00
Recall: 1.00

Confusion Matrix:
[[1380    2]
 [   0   28]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1382
           1       0.93      1.00      0.97        28

    accuracy                           1.00      1410
   macro avg       0.97      1.00      0.98      1410


In [10]:
# Evaluasi model dengan cross-validation
print("\n=== Evaluasi Model dengan Cross-Validation ===")
for model_name, model in resampled_models.items():
    # Cross-validation dengan StratifiedKFold
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=cv, scoring='accuracy')

    # Tampilkan hasil cross-validation
    print(f"\n{model_name} - Cross-Validation Accuracy: {scores.mean():.2f} (±{scores.std():.2f})")

    # Latih model dengan data resample
    model.fit(X_train_resampled, y_train_resampled)

    # Evaluasi model pada data testing
    evaluate_model(model, X_test, y_test)


=== Evaluasi Model dengan Cross-Validation ===

Random Forest (Resampled) - Cross-Validation Accuracy: 1.00 (±0.00)

Akurasi: 1.00
F1-Score: 1.00
Precision: 1.00
Recall: 1.00

Confusion Matrix:
[[1382    0]
 [   0   28]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1382
           1       1.00      1.00      1.00        28

    accuracy                           1.00      1410
   macro avg       1.00      1.00      1.00      1410
weighted avg       1.00      1.00      1.00      1410


Logistic Regression (Resampled) - Cross-Validation Accuracy: 1.00 (±0.00)

Akurasi: 1.00
F1-Score: 1.00
Precision: 1.00
Recall: 1.00

Confusion Matrix:
[[1380    2]
 [   0   28]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1382
           1       0.93      1.00      0.97        28

    accuracy                           1.00      1410
   ma

 **Alasan Melakukan Langkah-Langkah Tersebut:**
- **Penanganan Imbalance Data dengan SMOTE:**

  Dataset memiliki ketidakseimbangan kelas yang signifikan, dengan kelas 0 sebanyak 5529 sampel dan kelas 1 hanya 111 sampel. Ketidakseimbangan ini dapat menyebabkan model cenderung mengabaikan kelas minoritas (1) dan hanya fokus pada kelas mayoritas (0).

  SMOTE (Synthetic Minority Oversampling Technique) digunakan untuk menyeimbangkan distribusi kelas dengan membuat sampel sintetis dari kelas minoritas. Setelah SMOTE, distribusi kelas menjadi seimbang (5529 sampel untuk kelas 0 dan 5529 sampel untuk kelas 1).

- **Melatih Model dengan Data Resample:**

  Setelah menyeimbangkan dataset, model dilatih ulang menggunakan data yang telah diresample. Hal ini dilakukan untuk memastikan model dapat mempelajari pola dari kedua kelas secara seimbang.

- **Evaluasi Ulang Model dengan Data Resample:**

  Evaluasi dilakukan untuk memeriksa performa model setelah penanganan imbalance data. Tujuannya adalah memastikan bahwa model tidak hanya akurat pada kelas mayoritas tetapi juga pada kelas minoritas.

- **Cross-Validation:**

  Cross-validation digunakan untuk memastikan bahwa performa model konsisten dan tidak overfitting. Dengan membagi data menjadi beberapa fold dan melatih model secara bergantian, kita dapat memastikan bahwa model generalizes well ke data baru.

## **d. Analisis Hasil Evaluasi Model Klasifikasi**

1. **Identifikasi Kelemahan Model:**

- **Random Forest:** Tidak ada kelemahan yang signifikan. Model ini mencapai performa sempurna pada semua metrik evaluasi (akurasi, precision, recall, dan F1-score) untuk kedua kelas. Confusion Matrix menunjukkan tidak ada kesalahan prediksi (0 false positives dan 0 false negatives).

- **Logistic Regression:** Meskipun akurasi sempurna (1.00), terdapat 2 false positives pada kelas 1. Ini terlihat dari Confusion Matrix, di mana 2 instance kelas 0 salah diprediksi sebagai kelas 1. Precision untuk kelas 1 adalah 0.93, yang berarti ada sedikit kesalahan dalam memprediksi kelas minoritas.

2. **Overfitting atau Underfitting:**

- **Random Forest:** Tidak ada indikasi overfitting karena performa pada data training dan testing sama-sama sempurna. Cross-validation juga menunjukkan akurasi yang konsisten (±0.00).

- **Logistic Regression:** Tidak ada indikasi overfitting atau underfitting. Performa pada data testing sangat baik, meskipun ada sedikit kesalahan pada kelas minoritas.

3. **Rekomendasi Tindakan Lanjutan:**

- **Untuk Random Forest:** Karena performanya sudah sempurna, tidak diperlukan tindakan lanjutan. Namun, jika dataset sangat besar, pertimbangkan untuk mengoptimalkan hyperparameter (seperti n_estimators atau max_depth) untuk mengurangi waktu komputasi.

- **Untuk Logistic Regression:** Coba Algoritma Lain, jika precision untuk kelas 1 perlu ditingkatkan, coba algoritma lain seperti Gradient Boosting atau XGBoost yang mungkin lebih baik dalam menangani ketidakseimbangan kelas. Ubah Bobot Kelas seperti menggunakan parameter class_weight dengan nilai yang lebih disesuaikan untuk memberikan penekanan lebih pada kelas minoritas Jika memungkinkan, kumpulkan lebih banyak data untuk kelas minoritas (1) untuk meningkatkan kualitas pembelajaran model.


**Kesimpulan:**

Random Forest adalah pilihan terbaik untuk dataset ini, dengan performa sempurna pada semua metrik evaluasi. Logistic Regression juga menunjukkan performa yang sangat baik, tetapi memerlukan sedikit perbaikan pada precision untuk kelas minoritas. Rekomendasi tindakan lanjutan termasuk mencoba algoritma lain, mengoptimalkan hyperparameter, atau mengumpulkan data tambahan untuk kelas minoritas jika hasil belum memuaskan.

In [11]:
# Prediksi menggunakan model yang sudah dilatih dan menggunakan SMOTE
y_pred_rf = rf_resampled_trained.predict(X_test)
y_pred_lr = log_reg_resampled_trained.predict(X_test)


# Menyimpan hasil prediksi ke dalam DataFrame
results = pd.DataFrame({
    'Aktual': y_test,  # Nilai target asli
    'Prediksi_RandomForest': y_pred_rf,  # Prediksi dari Random Forest
    'Prediksi_LogisticRegression': y_pred_lr  # Prediksi dari Logistic Regression
})

# Jika ingin menyertakan fitur-fitur asli, gabungkan dengan DataFrame hasil prediksi
results_with_features = pd.concat([X_test.reset_index(drop=True), results], axis=1)

# Simpan hasil ke file CSV
results.to_csv('hasil_klasifikasi.csv', index=False)  # Hanya menyimpan prediksi
results_with_features.to_csv('hasil_klasifikasi_dengan_fitur.csv', index=False)  # Menyimpan prediksi dan fitur asli

print("Hasil klasifikasi telah disimpan ke file CSV.")

Hasil klasifikasi telah disimpan ke file CSV.
